In [15]:
!pip install numpy nltk


     |████████████████████████████████| 1.5MB 7.9MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.4.5-cp35-none-any.whl size=1449906 sha256=0df51d8681a0b89f73f14d79af346b46d445f2bbb73bee79c75486d702ff21b2
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk


In [4]:
# python3.5 buildtagger.py <train_file_absolute_path> <model_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def train_model(train_file, model_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    train_file = sys.argv[1]
    model_file = sys.argv[2]
    start_time = datetime.datetime.now()
    train_model(train_file, model_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


Finished...
Time: 0:00:00.000263


In [5]:
# python3.5 runtagger.py <test_file_absolute_path> <model_file_absolute_path> <output_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def tag_sentence(test_file, model_file, out_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    test_file = sys.argv[1]
    model_file = sys.argv[2]
    out_file = sys.argv[3]
    start_time = datetime.datetime.now()
    tag_sentence(test_file, model_file, out_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


IndexError: list index out of range

In [43]:
%%time

emission_dict = {}
transition_dict = {}

with open('sents.train') as training_file:
    word_tag_pairs = training_file.read().split(" ")
    word_count = len(word_tag_pairs)
    previous_tag = word_tag_pairs[0].split("/")[1]
    
    for word_tag in word_tag_pairs:
        
        word, tag = word_tag.split("/")[:2]
        
        if word_tag not in emission_dict.keys():
            emission_dict[word_tag] = 1
        else:
            emission_dict[word_tag] += 1
            
        if previous_tag + " " + tag not in transition_dict.keys():
            transition_dict[previous_tag + " " + tag] = 1
        else:
            transition_dict[previous_tag + " " + tag] += 1
        

CPU times: user 1.42 s, sys: 23 ms, total: 1.45 s
Wall time: 1.45 s


In [44]:
transition_dict 

{'IN .\nChicagoans': 1,
 'IN .\nAssets': 6,
 'IN .\nSocial': 2,
 'IN .\nHogs': 1,
 'IN .\nBlockbuster': 2,
 'IN .\nLazard': 1,
 'IN .\nBanks': 8,
 'IN VBN': 19790,
 'IN .\nSaudi': 3,
 'IN .\nUnilever': 2,
 'IN .\nInstitutions': 3,
 'IN .\nLast': 146,
 'IN .\nNeedless': 1,
 'IN .\nRoche': 2,
 'IN .\nPRIMERICA': 1,
 'IN .\nThrifts': 2,
 'IN .\nMore': 53,
 'IN .\nAustralia': 2,
 'IN .\nForty': 1,
 'IN .\nNobody': 8,
 'IN .\nCommercial': 8,
 'IN .\nStock-index': 7,
 'IN .\nRegrettably': 1,
 'IN .\nPresidents': 1,
 'IN :\nIs': 1,
 'IN Eagle': 5,
 'IN .\nOPPENHEIMER': 1,
 'IN .\nMachinists': 2,
 'IN .\nBeech': 1,
 'IN .\nStructural': 1,
 'IN .\nBlock': 1,
 'IN .\nPrivate-property': 1,
 'IN .\nWarehouse': 1,
 'IN .\nBulgaria': 1,
 'IN .\nTraders': 52,
 'IN .\nTimken': 1,
 'IN .\nHers': 1,
 'IN .\nAdded': 3,
 'IN .\nGrace': 3,
 'IN .\nUnconstitutional': 1,
 'IN .\n7:13': 1,
 'IN .\nQuantum': 4,
 'IN .\nBridge': 2,
 'IN .\nBob': 5,
 'IN .\nNTT': 1,
 'IN .\nGerald': 5,
 'IN .\nCAPITAL': 1,
 'IN 

In [23]:
import nltk 
nltk.download("tagsets")
tagset = nltk.help.upenn_tagset()



$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [24]:
print(tagset)

None
